In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Flatten,Conv1D,MaxPooling1D
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.optimizers import SGD, Adam
from keras.layers import Input, Dense
from keras.models import Model
from sklearn.metrics import confusion_matrix

Using TensorFlow backend.


In [2]:
data= np.loadtxt('norm.csv', delimiter=',', dtype='float32')
label=np.loadtxt('encodedlabel.csv',delimiter=',',dtype='float32')

In [3]:
train_data=data[0:900,:]
test_data=data[900:,:]
train_label=label[0:900,:]
test_label=label[900:,:]
print(train_data.shape, train_label.shape,test_data.shape, test_label.shape)

((900, 14), (900, 3), (232, 14), (232, 3))


In [4]:
input1=Input(shape=(14,1))
enc1=Dense(10,activation='relu')(input1)

In [11]:
# layer 1
input_1= Input(shape=(14,))
enc_1= Dense(7, activation='relu')(input_1)
dec_1=Dense(14, activation='relu')(enc_1)

autoenc_1= Model(input_1, dec_1)
encoder_1= Model(input_1, enc_1)

autoenc_1.compile(optimizer='adam', loss='mean_squared_error')
autoenc_1.fit(train_data,train_data,epochs=1,validation_split=0.2)


ly1_predict= encoder_1.predict(train_data)
print(ly1_predict.shape)

Train on 720 samples, validate on 180 samples
Epoch 1/1
720/720 [==============================] - 0s 489us/step - loss: 0.1013 - val_loss: 0.1054
(900, 7)


In [12]:
# layer 2
input_2= Input(shape=(7,))
enc_2= Dense(5, activation='relu')(input_2)
dec_2=Dense(7, activation='relu')(enc_2)

autoenc_2= Model(input_2, dec_2)
encoder_2= Model(input_2, enc_2)

autoenc_2.compile(optimizer='adam', loss='mean_squared_error')
autoenc_2.fit(ly1_predict,ly1_predict,epochs=1,validation_split=0.2)


ly2_predict= encoder_2.predict(ly1_predict)
print(ly2_predict.shape)

Train on 720 samples, validate on 180 samples
Epoch 1/1
720/720 [==============================] - 0s 535us/step - loss: 0.0625 - val_loss: 0.0787
(900, 5)


In [13]:
# layer 3
input_3= Input(shape=(5,))
enc_3= Dense(3, activation='relu')(input_3)
dec_3=Dense(5, activation='relu')(enc_3)

autoenc_3= Model(input_3, dec_3)
encoder_3= Model(input_3, enc_3)

autoenc_3.compile(optimizer='adam', loss='mean_squared_error')
autoenc_3.fit(ly2_predict,ly2_predict,epochs=1,validation_split=0.2)


ly3_predict= encoder_3.predict(ly2_predict)
print(ly3_predict.shape)

Train on 720 samples, validate on 180 samples
Epoch 1/1
720/720 [==============================] - 0s 538us/step - loss: 0.0101 - val_loss: 0.0080
(900, 3)


In [15]:
# Deep stacked auto encoder
ly_1= autoenc_1.layers[1].get_weights()
ly_2= autoenc_2.layers[1].get_weights()
ly_3= autoenc_3.layers[1].get_weights()

inputSAE= Input(shape=(14,))
hid_1= Dense(7, activation='relu', input_shape=(14,), name="layer1")(inputSAE)
hid_2= Dense(5, activation='relu',name="layer2")(hid_1)
hid_3= Dense(3, activation='softmax',)(hid_2)
hid_4= Dense(5, activation='relu',)(hid_3)
hid_5= Dense(7, activation='relu',)(hid_4)
hid_6= Dense(14, activation='relu',)(hid_5)

model= Model(inputSAE, hid_6)
SAE= Model(inputSAE, hid_3)

model.layers[1].set_weights(ly_1) # first dense layer
model.layers[2].set_weights(ly_2)
model.layers[3].set_weights(ly_3)


model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
model.fit(train_data, train_data,epochs=50,shuffle=True,verbose=1, validation_split=0.2)

SAE.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
SAE.fit(train_data, train_label,epochs=25,shuffle=True,verbose=1, validation_split=0.2)
scores=SAE.evaluate(test_data,test_label)
print("Accuracy:",scores[1]*100)
pred=SAE.predict(test_data)
cm=confusion_matrix(test_label.argmax(axis=1),pred.argmax(axis=1))
print("Confusion Matrix")
print(cm)
print('Precision(Normal)',(np.float(cm[0][0])/np.float(np.sum(cm[:,0]))))
print('Precision(DOS)',(np.float(cm[1][1])/np.float(np.sum(cm[:,1]))))
print('Precision(Probe)',(np.float(cm[2][2])/np.float(np.sum(cm[:,2]))))

Train on 720 samples, validate on 180 samples
Epoch 1/50
720/720 [==============================] - 1s 819us/step - loss: 0.1167 - acc: 0.0653 - val_loss: 0.1223 - val_acc: 0.5556
Epoch 2/50
720/720 [==============================] - 0s 132us/step - loss: 0.1075 - acc: 0.5833 - val_loss: 0.1081 - val_acc: 0.5556
Epoch 3/50
720/720 [==============================] - 0s 131us/step - loss: 0.0893 - acc: 0.5528 - val_loss: 0.0906 - val_acc: 0.1333
Epoch 4/50
720/720 [==============================] - 0s 122us/step - loss: 0.0754 - acc: 0.1556 - val_loss: 0.0804 - val_acc: 0.1333
Epoch 5/50
720/720 [==============================] - 0s 122us/step - loss: 0.0682 - acc: 0.1542 - val_loss: 0.0759 - val_acc: 0.1333
Epoch 6/50
720/720 [==============================] - 0s 127us/step - loss: 0.0653 - acc: 0.1625 - val_loss: 0.0742 - val_acc: 0.2944
Epoch 7/50
720/720 [==============================] - 0s 121us/step - loss: 0.0642 - acc: 0.5111 - val_loss: 0.0736 - val_acc: 0.5556
Epoch 8/50
720/7

720/720 [==============================] - 0s 130us/step - loss: 0.0748 - acc: 0.8694 - val_loss: 0.1030 - val_acc: 0.8167
Epoch 12/25
720/720 [==============================] - 0s 117us/step - loss: 0.0708 - acc: 0.8736 - val_loss: 0.0980 - val_acc: 0.8167
Epoch 13/25
720/720 [==============================] - 0s 110us/step - loss: 0.0670 - acc: 0.8764 - val_loss: 0.0929 - val_acc: 0.8167
Epoch 14/25
720/720 [==============================] - 0s 136us/step - loss: 0.0631 - acc: 0.8764 - val_loss: 0.0875 - val_acc: 0.8167
Epoch 15/25
720/720 [==============================] - 0s 129us/step - loss: 0.0592 - acc: 0.8778 - val_loss: 0.0806 - val_acc: 0.8167
Epoch 16/25
720/720 [==============================] - 0s 115us/step - loss: 0.0549 - acc: 0.8778 - val_loss: 0.0722 - val_acc: 0.8167
Epoch 17/25
720/720 [==============================] - 0s 125us/step - loss: 0.0497 - acc: 0.8875 - val_loss: 0.0632 - val_acc: 0.8667
Epoch 18/25
720/720 [==============================] - 0s 103us/ste